# 📩 Building a Spam Classifier with Foundation Models

## 🚀 Project Overview

Foundation models enable a wide range of intelligent applications without requiring task-specific training.  
In this project, you will build a **spam classifier using only prompt engineering**, leveraging the model’s language understanding and reasoning capabilities.

The objective is to classify SMS messages as **SPAM** or **NOT SPAM** while enforcing a structured **JSON output**, making the results easy to parse and integrate into downstream applications.

---

## 🧭 Step-by-Step Workflow

### 🧩 Step 1 — Define the Task and Output Format  
- Create a **SYSTEM_PROMPT** that clearly specifies:
  - 🤖 The model’s role  
  - 🎯 The classification task  
  - 📦 The required JSON output structure  
- This ensures consistent and machine-readable responses.

---

### 🎬 Step 2 — Zero-Shot Classification  
- Start with **instructions only**, without examples.  
- Evaluate how well the model performs based solely on its prior knowledge.

---

### 📘 Step 3 — Few-Shot Enhancement  
- Add a small set of **labeled examples** to the prompt.  
- Use these in-context demonstrations to improve prediction consistency and accuracy.

---

### 📱 Step 4 — Apply to Spam Detection  
- Use the prompt structure to classify SMS messages as **SPAM** or **NOT SPAM**.  
- Validate the JSON output and analyze prediction quality.

---

### 🚀 Step 5 — Iterate and Improve  
- Refine prompt wording and constraints.  
- Experiment with different example selections.  
- Compare zero-shot and few-shot performance.  
- Identify edge cases and limitations.

---

## ✅ Key Takeaways

- 🛠️ Well-designed prompts can transform foundation models into practical classifiers.  
- ⚡ Zero-shot prompting is fast and simple but may lack precision.  
- 🎯 Few-shot prompting often improves reliability but does not eliminate ambiguity.  
- 🔁 The same prompting pattern can be adapted to many classification tasks beyond spam detection.


# 📚 Importing libraries and environment variables

In [ ]:
import litellm
import os

if os.getenv("OPENAI_API_KEY"):
    litellm.openai_key = os.getenv("OPENAI_API_KEY")

# If using Vocareum, you can also set your API key here directly
# Uncomment and replace the string with your actual Vocareum API key

if (litellm.openai_key or "").startswith("voc-"):
    litellm.api_base = "https://openai.vocareum.com/v1"
    print("Using Vocareum OpenAI API base URL")

Using Vocareum OpenAI API base URL


## 📊 Data Preparation

To build and evaluate the spam classifier, you need a dataset of messages labeled as **SPAM** or **NOT SPAM**.  
Selecting a representative dataset is essential to capture a diverse range of message patterns, including both legitimate and unwanted content.

A well-balanced and well-curated dataset improves the reliability of the evaluation and helps ensure that the classifier generalizes effectively to real-world scenarios.


In [3]:
# Load the sms_spam dataset using the datasets library
# No changes needed in this cell

from datasets import load_dataset

dataset = load_dataset("sms_spam", split=["train"])[0]

for entry in dataset.select(range(3)):
    sms = entry["sms"]
    label = entry["label"]
    print(f"label={label}, sms={sms}")

label=0, sms=Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

label=0, sms=Ok lar... Joking wif u oni...

label=1, sms=Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's



## 🏷️ Improving Label Readability

The numerical labels can be difficult to interpret at a glance.  
To make the output more readable and user-friendly, we will create helper functions to convert numeric IDs into meaningful labels.


In [ ]:
id2label = {0: "NOT SPAM", 1: "SPAM"}
label2id = {"NOT SPAM": 0, "SPAM": 1}

for entry in dataset.select(range(3)):
    sms = entry["sms"]
    label_id = entry["label"]
    print(f"label={id2label[label_id]}, sms={sms}")

label=NOT SPAM, sms=Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

label=NOT SPAM, sms=Ok lar... Joking wif u oni...

label=SPAM, sms=Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's



## 🤖 Build and Evaluate the Spam Classifier

Using the foundation model and the prepared dataset, you will build a spam classifier based entirely on prompting.

Create a prompt that asks the model to classify 15 messages as either **"SPAM"** or **"NOT SPAM"**.  
To simplify downstream processing and automatic validation, require the LLM to return the predictions in a structured **JSON format**.


In [6]:
# Create a helper function to get multiple sms messages as a single string
# No changes needed in this cell


def get_sms_messages_string(dataset, item_numbers):
    sms_messages_string = ""
    for item_number, entry in zip(item_numbers, dataset.select(item_numbers)):
        sms = entry["sms"]
        label_id = entry["label"]

        sms_messages_string += f"{item_number} -> {sms}\n"

    return sms_messages_string


print(get_sms_messages_string(dataset, [0, 1, 2]))

0 -> Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

1 -> Ok lar... Joking wif u oni...

2 -> Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's




## 🧩 Prompt Construction

Next, write the code that generates the prompt sent to the LLM.  
The prompt should include a small set of SMS messages to be classified, along with clear instructions describing the task.

Many LLMs can also format their responses automatically when instructed. For example, you can explicitly request: **"Respond in JSON format."** This helps ensure consistent and machine-readable outputs.


In [ ]:
# Get a few messages and format them as a string
sms_messages_string = get_sms_messages_string(dataset, range(7, 15))

# The input should be of the form
# 11 -> ...
# 16 -> ...
# 23 -> ...

# The output should be of the form
# {
#     "11": "NOT SPAM",
#     "16": "SPAM",
#     "23": "NOT SPAM"
# }

SYSTEM_PROMPT = """ You are a text classification assistant.
You will receive multiple SMS messages in the format:
<id> -> <message>

Your task is to classify each message as either "SPAM" or "NOT SPAM".

Return the result strictly in valid JSON format, where:
- The keys are the message IDs.
- The values are the predicted labels ("SPAM" or "NOT SPAM").
- Do not include any extra text, explanations, or formatting outside the JSON.
"""

USER_PROMPT = sms_messages_string


print("SYSTEM PROMPT:")
print(SYSTEM_PROMPT)
print("\nUSER PROMPT:")
print(USER_PROMPT)


SYSTEM PROMPT:
 You are a text classification assistant.
You will receive multiple SMS messages in the format:
<id> -> <message>

Your task is to classify each message as either "SPAM" or "NOT SPAM".

Return the result strictly in valid JSON format, where:
- The keys are the message IDs.
- The values are the predicted labels ("SPAM" or "NOT SPAM").
- Do not include any extra text, explanations, or formatting outside the JSON.


USER PROMPT:
7 -> As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune

8 -> WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.

9 -> Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030

10 -> I'm gonna be home soon and i don't want to talk about thi

In [ ]:
from litellm import completion

response = completion(
    model="gpt-5-nano",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
response = response.choices[0].message.content
print(response)

# Check that response is in valid JSON format
try:
    import json

    json.loads(response)
    print("Response is valid JSON")
except json.JSONDecodeError:
    print("Response is not valid JSON")


{
  "7": "SPAM",
  "8": "SPAM",
  "9": "SPAM",
  "10": "NOT SPAM",
  "11": "SPAM",
  "12": "SPAM",
  "13": "NOT SPAM",
  "14": "NOT SPAM"
}
Response is valid JSON


## 📊 Performance Evaluation

To quantitatively assess the quality of the predictions, we implement a function that matches the LLM outputs against the reference labels in the dataset.  
The resulting accuracy metric provides a simple measure of classification performance.


In [ ]:
def get_accuracy(response, dataset, original_indices):
    correct = 0
    total = 0

    if isinstance(response, str):
        import json

        try:
            response = json.loads(response)
        except json.JSONDecodeError as e:
            print("Error decoding JSON response:", e)
            return

    for entry_number, prediction in response.items():
        if int(entry_number) not in original_indices:
            continue

        label_id = dataset[int(entry_number)]["label"]
        label = id2label[label_id]

        # If the prediction from the LLM matches the label in the dataset
        # we increment the number of correct predictions.
        # (Since LLMs do not always produce the same output, we use the
        # lower case version of the strings for comparison)
        if prediction.lower() == label.lower():
            correct += 1
        else:
            print(
                f"Mismatch for entry {entry_number}: predicted={prediction}, actual={label}"
            )

        # increment the total number of predictions
        total += 1

    try:
        accuracy = correct / total
    except ZeroDivisionError:
        print("No matching results found!")
        return

    return round(accuracy, 2)


print(f"Accuracy: {get_accuracy(response, dataset, range(7, 15))}")

Mismatch for entry 7: predicted=SPAM, actual=NOT SPAM
Accuracy: 0.88


That's a strong result — and even better if all predictions were correct.

While the model may not classify every possible example perfectly, this performance is an excellent starting point, especially given that no labeled examples or training data were provided.

The results show that the model can reliably distinguish between spam and legitimate messages with a high level of accuracy, illustrating how foundation models can be effectively used to build practical spam classification systems using only prompting.


## 🚀 Build an Improved Classifier

Providing the LLM with a small set of labeled examples can often improve its performance.  
In this step, we apply few-shot prompting to guide the model with concrete examples and evaluate whether this leads to more accurate and consistent classifications.

In [ ]:
def get_few_shot_examples_string(dataset, item_numbers):
    examples_string = ""

    examples_string += "EXAMPLE INPUT:\n"
    for item_number, entry in zip(item_numbers, dataset.select(item_numbers)):
        sms = entry["sms"]

        examples_string += f"{item_number} -> {sms}\n"

    examples_string += "EXAMPLE OUTPUT:\n"
    examples_string += "{\n"
    for item_number, entry in zip(item_numbers, dataset.select(item_numbers)):
        label_id = entry["label"]
        label = id2label[label_id]
        examples_string += f'    "{item_number}": "{label}",\n'
    examples_string += "}\n"

    return examples_string


print(get_few_shot_examples_string(dataset, range(54, 60)))

EXAMPLE INPUT:
54 -> SMS. ac Sptv: The New Jersey Devils and the Detroit Red Wings play Ice Hockey. Correct or Incorrect? End? Reply END SPTV

55 -> Do you know what Mallika Sherawat did yesterday? Find out now @  &lt;URL&gt;

56 -> Congrats! 1 year special cinema pass for 2 is yours. call 09061209465 now! C Suprman V, Matrix3, StarWars3, etc all 4 FREE! bx420-ip4-5we. 150pm. Dont miss out! 

57 -> Sorry, I'll call later in meeting.

58 -> Tell where you reached

59 -> Yes..gauti and sehwag out of odi series.

EXAMPLE OUTPUT:
{
    "54": "SPAM",
    "55": "NOT SPAM",
    "56": "SPAM",
    "57": "NOT SPAM",
    "58": "NOT SPAM",
    "59": "NOT SPAM",
}



In [ ]:
# Get a few messages and format them as a string
sms_messages_string = get_sms_messages_string(dataset, range(7, 15))

# Construct a SYSTEM_PROMPT and USER_PROMPT to send to the LLM. This time, let's include the
# labelled examples to see if it improves accuracy. Hint: use get_few_shot_examples_string(dataset, range(54, 60))

SYSTEM_PROMPT = """
You are a text classification assistant.

You will receive:
1) Some labelled SMS examples.
2) A list of unlabelled SMS messages.

Your task is to classify each unlabelled message as either "SPAM" or "NOT SPAM".

Return the result strictly in valid JSON format:
- Keys must be the message IDs.
- Values must be "SPAM" or "NOT SPAM".
- Do not include explanations or any text outside the JSON.
"""

USER_PROMPT = f"""
Here are some labelled examples:

{get_few_shot_examples_string(dataset, range(54, 60))}

Now classify the following unlabelled messages:

{sms_messages_string}
"""



print("SYSTEM_PROMPT:")
print(SYSTEM_PROMPT)
print("USER_PROMPT:")
print(USER_PROMPT)

SYSTEM_PROMPT:

You are a text classification assistant.

You will receive:
1) Some labelled SMS examples.
2) A list of unlabelled SMS messages.

Your task is to classify each unlabelled message as either "SPAM" or "NOT SPAM".

Return the result strictly in valid JSON format:
- Keys must be the message IDs.
- Values must be "SPAM" or "NOT SPAM".
- Do not include explanations or any text outside the JSON.

USER_PROMPT:

Here are some labelled examples:

EXAMPLE INPUT:
54 -> SMS. ac Sptv: The New Jersey Devils and the Detroit Red Wings play Ice Hockey. Correct or Incorrect? End? Reply END SPTV

55 -> Do you know what Mallika Sherawat did yesterday? Find out now @  &lt;URL&gt;

56 -> Congrats! 1 year special cinema pass for 2 is yours. call 09061209465 now! C Suprman V, Matrix3, StarWars3, etc all 4 FREE! bx420-ip4-5we. 150pm. Dont miss out! 

57 -> Sorry, I'll call later in meeting.

58 -> Tell where you reached

59 -> Yes..gauti and sehwag out of odi series.

EXAMPLE OUTPUT:
{
    "54":

📥 Paste in your response from the LLM below:

In [ ]:
# Get the response from the LLM

from litellm import completion

response = completion(
    model="gpt-5-nano",
    messages=[
        {"role": "user", "content": USER_PROMPT},
        {"role": "system", "content": SYSTEM_PROMPT},
    ],
)

response = response.choices[0].message.content
print(response)

# Check that response is in valid JSON format
try:
    import json

    json.loads(response)
    print("Response is valid JSON")
except json.JSONDecodeError:
    print("Response is not valid JSON")

{
  "7": "NOT SPAM",
  "8": "SPAM",
  "9": "SPAM",
  "10": "NOT SPAM",
  "11": "SPAM",
  "12": "SPAM",
  "13": "NOT SPAM",
  "14": "NOT SPAM"
}
Response is valid JSON


Let's check the accuracy now

In [ ]:
# Estimate the accuracy of your classifier by comparing your responses to the labels in the dataset

print(f"Accuracy: {get_accuracy(response, dataset, range(7, 15)):.2f}")

Accuracy: 1.00


Do you still observe any errors in the results? If so, what factors do you believe are contributing to these errors?


<br /><br /><br /><br /><br /><br /><br /><br /><br />